In [1]:
import torchvision.models as models

In [2]:
model = models.mobilenet_v3_small(pretrained=True)
print(model)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100.0%


MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride

In [1]:
import torch
from torchvision import transforms
from torchvision.models import mobilenet_v3_small
from PIL import Image
import time
import datetime
import pytz

def read_classes():
    """
    Load the ImageNet class names.
    """
    with open("imagenet-classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    return categories

print("Before loading model: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Load the pre-trained Mobilenetv3 model
model = mobilenet_v3_small(pretrained=True)
model.eval()  # Set the model to evaluation mode
categories = read_classes()

Before loading model:  2024-10-18 17:57:17.025792+01:00


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100.0%


In [19]:
# Load and preprocess an example image
print("Loading image: ",datetime.datetime.now(pytz.timezone('Europe/London')))
image_path = 'dog_1.jpg'
input_image = Image.open(image_path).convert('RGB')

Loading image:  2024-10-18 18:08:15.296827+01:00


In [20]:
print("Image transformation: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Define the image transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to the input image
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension


Image transformation:  2024-10-18 18:08:18.284938+01:00


In [21]:
# Make the prediction
with torch.no_grad():
    start_time = time.time()
    print("During prediction: ", datetime.datetime.now(pytz.timezone('Europe/London')))
    # print("During prediction: ",jtop().stats)
    output = model(input_batch)
    end_time = time.time()

# Get the predicted class index
# get the softmax probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)
# check the top category that are predicted
top5_prob, top5_catid = torch.topk(probabilities, 1)

# Print the predicted class label and inference time
inference_time = end_time - start_time
print(f'Inference time: {inference_time:.4f} seconds')
print(f"Accuracy: {top5_prob[0].item()*100:.3f}%")
print(f"Predicted class:{categories[top5_catid[0]]}")      

During prediction:  2024-10-18 18:08:22.015930+01:00
Inference time: 2.6493 seconds
Accuracy: 81.686%
Predicted class:Newfoundland, Newfoundland dog


In [5]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total trainable parameters: {total_params}')

Total trainable parameters: 2542856


In [6]:
import torch
import torchvision.models as models

# Load the pre-trained Mobilenetv3 model
model = mobilenet_v3_small(pretrained=True)

# Define input size (224x224x3) for Mobilenetv3
input_size = (224, 224, 3)

# Function to calculate FLOPs for the model
def count_flops(model, input_size):
    # Define a tensor of appropriate size
    input_tensor = torch.randn(1, *input_size)
    
    # Switch to evaluation mode
    model.eval()
    
    # Move model to appropriate device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    model.to(device)
    
    # Iterate through model's layers
    flops = 0
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            # For convolutional layers
            output_size = (input_size[0] - module.kernel_size[0] + 2 * module.padding[0]) // module.stride[0] + 1
            output_size = (output_size - module.kernel_size[1] + 2 * module.padding[1]) // module.stride[1] + 1
            flops += module.in_channels * module.out_channels * module.kernel_size[0] * module.kernel_size[1] * output_size * output_size
            input_size = (output_size, output_size, module.out_channels)
        elif isinstance(module, torch.nn.MaxPool2d):
            # For max pooling layers
            output_size = (input_size[0] - module.kernel_size) // module.stride + 1
            flops += input_size[2] * output_size * output_size
            input_size = (output_size, output_size, input_size[2])
        elif isinstance(module, torch.nn.Linear):
            # For fully connected layers
            flops += module.in_features * module.out_features
            input_size = (module.out_features,)
    
    return flops

# Calculate FLOPs
total_flops = count_flops(model, input_size)
print("Total FLOPs:", total_flops)

Total FLOPs: 29234304


Total FLOPs: 29234304
Before loading model:  2024-10-18 17:57:17.025792+01:00

0: 1.jpg

Loading image:  2024-10-18 17:57:31.158511+01:00
Image transformation:  2024-10-18 17:57:35.574077+01:00
During prediction:  2024-10-18 17:57:39.639212+01:00
Inference time: 2.9779 seconds
Accuracy: 40.332%
Predicted class:Italian greyhound

datetime.datetime(2024, 10, 18, 17, 57, 17, 948191)
datetime.datetime(2024, 10, 18, 17, 57, 18, 950298)
datetime.datetime(2024, 10, 18, 17, 57, 19, 952551)
datetime.datetime(2024, 10, 18, 17, 57, 20, 942129)
datetime.datetime(2024, 10, 18, 17, 57, 21, 943182)
datetime.datetime(2024, 10, 18, 17, 57, 22, 943864)
datetime.datetime(2024, 10, 18, 17, 57, 23, 957899)
datetime.datetime(2024, 10, 18, 17, 57, 24, 966308)
datetime.datetime(2024, 10, 18, 17, 57, 25, 954725)
datetime.datetime(2024, 10, 18, 17, 57, 26, 951080)
datetime.datetime(2024, 10, 18, 17, 57, 27, 965039)
datetime.datetime(2024, 10, 18, 17, 57, 28, 964528)
datetime.datetime(2024, 10, 18, 17, 57, 29, 970994)
datetime.datetime(2024, 10, 18, 17, 57, 30, 957246)
datetime.datetime(2024, 10, 18, 17, 57, 31, 966759)
datetime.datetime(2024, 10, 18, 17, 57, 32, 969495)
datetime.datetime(2024, 10, 18, 17, 57, 33, 959996)
datetime.datetime(2024, 10, 18, 17, 57, 34, 972118)
datetime.datetime(2024, 10, 18, 17, 57, 35, 974874)
datetime.datetime(2024, 10, 18, 17, 57, 36, 975562)
datetime.datetime(2024, 10, 18, 17, 57, 37, 976466)
datetime.datetime(2024, 10, 18, 17, 57, 38, 966991)
datetime.datetime(2024, 10, 18, 17, 57, 40, 053279)

Average CPU Temperature: 31.13°C
Average GPU Temperature: 32.02°C
Average Power CPU: 868.96mW
Average Power GPU: 868.96mW
Average Power TOT: 2325.17mW
Average RAM: 0.4688

{'Avg_CPU_temp': 31.130434782608695, 'Avg_GPU_temp': 32.02173913043478, 'Avg_Power_CPU': 868.9565217391304, 'Avg_Power_GPU': 868.9565217391304, 'Avg_CPU_RAM': 0.46881048197927816}

1: cat_0

Loading image:  2024-10-18 17:59:21.789642+01:00
Image transformation:  2024-10-18 17:59:24.607439+01:00
During prediction:  2024-10-18 17:59:27.012594+01:00
Inference time: 2.8112 seconds
Accuracy: 14.485%
Predicted class:water jug

datetime.datetime(2024, 10, 18, 17, 59, 21, 120204)
datetime.datetime(2024, 10, 18, 17, 59, 22, 126424)
datetime.datetime(2024, 10, 18, 17, 59, 23, 124276)
datetime.datetime(2024, 10, 18, 17, 59, 24, 117592)
datetime.datetime(2024, 10, 18, 17, 59, 25, 121873)
datetime.datetime(2024, 10, 18, 17, 59, 26, 135385)
datetime.datetime(2024, 10, 18, 17, 59, 27, 196186)
datetime.datetime(2024, 10, 18, 17, 59, 28, 133128)
datetime.datetime(2024, 10, 18, 17, 59, 29, 181014)

Average CPU Temperature: 31.61°C
Average GPU Temperature: 32.50°C
Average Power CPU: 1231.00mW
Average Power GPU: 1231.00mW
Average Power TOT: 2643.22mW
Average RAM: 0.4709
{'Avg_CPU_temp': 31.61111111111111, 'Avg_GPU_temp': 32.5, 'Avg_Power_CPU': 1231.0, 'Avg_Power_GPU': 1231.0, 'Avg_CPU_RAM': 0.4708981101005223}

2: cat_1
Loading image:  2024-10-18 18:01:04.291168+01:00
Image transformation:  2024-10-18 18:01:06.273660+01:00
During prediction:  2024-10-18 18:01:09.836196+01:00
Inference time: 2.8233 seconds
Accuracy: 53.589%
Predicted class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 18, 01, 04, 259825)
datetime.datetime(2024, 10, 18, 18, 01, 05, 279781)
datetime.datetime(2024, 10, 18, 18, 01, 06, 262401)
datetime.datetime(2024, 10, 18, 18, 01, 07, 274778)
datetime.datetime(2024, 10, 18, 18, 01, 08, 276707)
datetime.datetime(2024, 10, 18, 18, 01, 09, 277619)
datetime.datetime(2024, 10, 18, 18, 01, 10, 316841)
datetime.datetime(2024, 10, 18, 18, 01, 11, 342538)

Average CPU Temperature: 31.69°C
Average GPU Temperature: 32.69°C
Average Power CPU: 1076.88mW
Average Power GPU: 1076.88mW
Average Power TOT: 2453.38mW
Average RAM: 0.4712
{'Avg_CPU_temp': 31.6875, 'Avg_GPU_temp': 32.6875, 'Avg_Power_CPU': 1076.875, 'Avg_Power_GPU': 1076.875, 'Avg_CPU_RAM': 0.47117206669228723}

3: dog_0

Loading image:  2024-10-18 18:03:41.256764+01:00
Image transformation:  2024-10-18 18:03:43.954394+01:00
During prediction:  2024-10-18 18:03:50.639748+01:00
Inference time: 2.9095 seconds
Accuracy: 69.595%
Predicted class:malamute, malemute, Alaskan malamute

datetime.datetime(2024, 10, 18, 18, 03, 41, 482930)
datetime.datetime(2024, 10, 18, 18, 03, 42, 480816)
datetime.datetime(2024, 10, 18, 18, 03, 43, 488559)
datetime.datetime(2024, 10, 18, 18, 03, 44, 482902)
datetime.datetime(2024, 10, 18, 18, 03, 45, 488933)
datetime.datetime(2024, 10, 18, 18, 03, 46, 492824)
datetime.datetime(2024, 10, 18, 18, 03, 47, 505770)
datetime.datetime(2024, 10, 18, 18, 03, 48, 501570)
datetime.datetime(2024, 10, 18, 18, 03, 49, 491754)
datetime.datetime(2024, 10, 18, 18, 03, 50, 501876)
datetime.datetime(2024, 10, 18, 18, 03, 51, 510798)
datetime.datetime(2024, 10, 18, 18, 03, 52, 532689)
datetime.datetime(2024, 10, 18, 18, 03, 53, 498738)

Average CPU Temperature: 31.58°C
Average GPU Temperature: 32.50°C
Average Power CPU: 1027.62mW
Average Power GPU: 1027.62mW
Average Power TOT: 2414.23mW
Average RAM: 0.4707
{'Avg_CPU_temp': 31.576923076923077, 'Avg_GPU_temp': 32.5, 'Avg_Power_CPU': 1027.6153846153845, 'Avg_Power_GPU': 1027.6153846153845, 'Avg_CPU_RAM': 0.4706899283154936}

4: cat_2

Loading image:  2024-10-18 18:05:06.948375+01:00
Image transformation:  2024-10-18 18:05:09.383007+01:00
During prediction:  2024-10-18 18:05:12.125914+01:00
Inference time: 2.8526 seconds
Accuracy: 99.132%
Predicted class:Persian cat

datetime.datetime(2024, 10, 18, 18, 05, 06, 602950)
datetime.datetime(2024, 10, 18, 18, 05, 07, 604978)
datetime.datetime(2024, 10, 18, 18, 05, 08, 603793)
datetime.datetime(2024, 10, 18, 18, 05, 09, 602987)
datetime.datetime(2024, 10, 18, 18, 05, 10, 610692)
datetime.datetime(2024, 10, 18, 18, 05, 11, 607423)
datetime.datetime(2024, 10, 18, 18, 05, 12, 657260)
datetime.datetime(2024, 10, 18, 18, 05, 13, 695027)
datetime.datetime(2024, 10, 18, 18, 05, 14, 633435)

Average CPU Temperature: 32.06°C
Average GPU Temperature: 32.72°C
Average Power CPU: 1279.44mW
Average Power GPU: 1279.44mW
Average Power TOT: 2683.11mW
Average RAM: 0.4706
{'Avg_CPU_temp': 32.05555555555556, 'Avg_GPU_temp': 32.72222222222222, 'Avg_Power_CPU': 1279.4444444444443, 'Avg_Power_GPU': 1279.4444444444443, 'Avg_CPU_RAM': 0.4706216489348307}

5: dog_1

Loading image:  2024-10-18 18:08:15.296827+01:00
Image transformation:  2024-10-18 18:08:18.284938+01:00
During prediction:  2024-10-18 18:08:22.015930+01:00
Inference time: 2.6493 seconds
Accuracy: 81.686%
Predicted class:Newfoundland, Newfoundland dog

datetime.datetime(2024, 10, 18, 18, 08, 15, 866883)
datetime.datetime(2024, 10, 18, 18, 08, 16, 864828)
datetime.datetime(2024, 10, 18, 18, 08, 17, 858168)
datetime.datetime(2024, 10, 18, 18, 08, 18, 875126)
datetime.datetime(2024, 10, 18, 18, 08, 19, 869052)
datetime.datetime(2024, 10, 18, 18, 08, 20, 874626)
datetime.datetime(2024, 10, 18, 18, 08, 21, 874836)
datetime.datetime(2024, 10, 18, 18, 08, 22, 882504)
datetime.datetime(2024, 10, 18, 18, 08, 23, 974059)
datetime.datetime(2024, 10, 18, 18, 08, 24, 877386)

Average CPU Temperature: 31.80°C
Average GPU Temperature: 32.60°C
Average Power CPU: 1156.60mW
Average Power GPU: 1156.60mW
Average Power TOT: 2546.70mW
Average RAM: 0.4706

{'Avg_CPU_temp': 31.8, 'Avg_GPU_temp': 32.6, 'Avg_Power_CPU': 1156.6, 'Avg_Power_GPU': 1156.6, 'Avg_CPU_RAM': 0.4705887801657453}